In [69]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [68]:
# Loading data
df_crop = pd.read_parquet("data/processed/crop_annual.parquet")
df_weather = pd.read_parquet("data/processed/weather_state_month.parquet")

# Checking dtypes for merge
print(df_crop.dtypes, '\n')
print(df_weather.dtypes)

state_alpha       object
year               int64
crop              object
yield             object
area_harvested    object
dtype: object 

state          object
year            int64
month           int64
avg_temp      float64
prcp          float64
n_stations      int64
temp_Z        float64
prcp_Z        float64
dtype: object


In [70]:
# Checking for missing values
print(df_crop.isna().sum(), '\n')
print(df_weather.isna().sum(), '\n')

# Listwise deletion of empty values
df_weather.dropna(inplace=True)

# Comparing year ranges
print(df_weather['year'].max(), df_weather['year'].min())
print(df_crop['year'].max(), df_crop['year'].min())

state_alpha       0
year              0
crop              0
yield             0
area_harvested    0
dtype: int64 

state          0
year           0
month          0
avg_temp      35
prcp           8
n_stations     0
temp_Z        35
prcp_Z         8
dtype: int64 

2026 1884
2025 1961


In [71]:
df_weather.columns

Index(['state', 'year', 'month', 'avg_temp', 'prcp', 'n_stations', 'temp_Z',
       'prcp_Z'],
      dtype='object')

In [72]:
# Restricting weather df year range
df_weather = df_weather[df_weather["year"].between(1961, 2025)]

# Aggregating for annual estimates
df_weather_yearly = df_weather.groupby(["state", "year"], as_index=False).agg(
        avg_temp_yr=("avg_temp", "mean"),
        prcp_yr=("prcp", "sum"),
        temp_Z_yr=("temp_Z", "mean"),
        prcp_Z_yr=("prcp_Z", "mean"),
        n_stations_yr=("n_stations", "mean"),
    ).copy()

In [73]:
# Merging weather on crop
df_panel = df_crop.merge(
    df_weather,
    left_on=["state_alpha","year"],
    right_on=["state","year"],
    how="inner"
)

# Analysis

Area harvested endogenous

In [66]:
df_panel

,state_alpha,year,crop,yield,area_harvested,state,avg_temp_yr,prcp_yr,temp_Z_yr,prcp_Z_yr,n_stations_yr
0,AL,1961,WHEAT,26,"56,000",AL,16.868161,1739.493333,-0.675835,28.759685,15.000000
1,AL,1962,WHEAT,24,"35,000",AL,17.433425,1286.320000,-0.110571,-9.004760,15.000000
2,AL,1963,WHEAT,23.5,"42,000",AL,16.702125,1216.718571,-0.841871,-14.804879,14.833333
3,AL,1964,WHEAT,25,"64,000",AL,17.145506,1633.945916,-0.398490,19.964066,13.833333
4,AL,1965,WHEAT,24.5,"55,000",AL,17.465072,1273.923810,-0.078925,-10.037776,14.416667
...,...,...,...,...,...,...,...,...,...,...,...
2674,WY,2021,WHEAT,32,"95,000",WY,7.084397,276.085913,0.849217,-4.245610,22.166667
2675,WY,2022,WHEAT,17,"95,000",WY,5.894840,295.559053,-0.340340,-2.622849,23.166667
2676,WY,2023,WHEAT,30,"90,000",WY,5.781864,404.096756,-0.453315,6.421960,23.333333
2677,WY,2024,WHEAT,31,"91,000",WY,7.450915,263.732769,1.215736,-5.275039,22.000000


In [74]:
w = df_weather.copy()

# Building wide tables of monthly instruments
tempZ = w.pivot_table(index=["state","year"], columns="month", values="temp_Z")
prcpZ = w.pivot_table(index=["state","year"], columns="month", values="prcp_Z")

# Giving names to columns based on month
tempZ.columns = [f"tempZ_m{int(m)}" for m in tempZ.columns]
prcpZ.columns = [f"prcpZ_m{int(m)}" for m in prcpZ.columns]

Zwide = tempZ.join(prcpZ).reset_index()

In [75]:
Zwide

,state,year,tempZ_m1,tempZ_m2,tempZ_m3,tempZ_m4,tempZ_m5,tempZ_m6,tempZ_m7,tempZ_m8,...,prcpZ_m3,prcpZ_m4,prcpZ_m5,prcpZ_m6,prcpZ_m7,prcpZ_m8,prcpZ_m9,prcpZ_m10,prcpZ_m11,prcpZ_m12
0,AL,1961,-3.197920,2.209975,1.202609,-2.204850,-1.669153,-2.050904,-1.099310,-1.154869,...,63.721351,-6.919014,-36.864447,65.777775,-17.691224,16.767785,-25.111338,-49.068100,24.515399,171.686542
1,AL,1962,-1.333368,4.148131,-2.478109,-1.569209,2.647645,-0.348043,0.381419,0.310272,...,-13.765316,28.200986,-78.811113,21.004442,-33.204557,-48.672215,14.368662,-4.528100,10.748732,-51.873458
2,AL,1963,-3.426218,-3.557254,1.668145,1.536983,0.547841,-0.287447,-1.151925,-0.282538,...,-20.485316,-19.592347,-19.657780,20.011108,12.102109,-44.438882,-20.164671,-73.218100,7.828732,-1.071554
3,AL,1964,-1.359461,-2.979757,-1.102255,1.341887,0.335248,0.651534,-0.891265,-0.738219,...,43.905160,150.740474,-35.458732,-40.824130,58.823776,3.532913,-18.743133,54.460471,7.195399,8.653208
4,AL,1965,1.036899,-0.986176,-2.304392,1.744010,1.152825,-0.953697,-0.711931,-0.597013,...,20.190875,-59.872347,-82.873018,60.268727,21.824967,5.387309,5.988662,0.705233,-45.360315,-40.900125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,WY,2021,1.678511,-3.622384,0.578136,-1.431581,-0.305844,3.481911,2.343117,0.043374,...,-4.864541,-9.084360,-11.491265,-27.654246,-9.154540,13.848147,-12.172818,15.503410,-9.902832,3.015919
3076,WY,2022,0.342806,-1.815428,0.606420,-2.538534,-1.389492,0.031471,1.612886,1.850630,...,-10.931207,-0.910447,8.862292,-14.049108,-14.489323,11.761190,0.855081,-7.243148,2.452784,4.194328
3077,WY,2023,-1.172258,-2.957430,-4.430004,-2.322875,2.016526,-0.591113,-0.133623,0.024153,...,-2.618164,-13.801027,-17.437708,54.642197,-1.920844,24.661190,10.431530,23.740564,-1.881093,-9.110563
3078,WY,2024,-0.567734,2.365781,0.704592,0.721691,-1.671981,1.896849,0.533008,0.839526,...,5.623338,-5.561633,-4.968538,-28.088445,-13.445844,4.134482,-13.439616,-5.054891,-5.589788,-6.224990


In [94]:
# Merging into panel
df_Z = df_panel.merge(Zwide, left_on=["state_alpha","year"], right_on=["state","year"], how="inner").copy()

# Need to strip commass from area harvested
df_Z["area_harvested"] = df_Z["area_harvested"].astype(str).str.replace(",", "")

# Turning to numeric
df_Z['area_harvested'] = pd.to_numeric(df_Z['area_harvested'], errors='coerce')
# Taking logarithm of area harvested
df_Z["log_area"] = np.log(df_Z["area_harvested"])

In [95]:
df_Z

,state_alpha,year,crop,yield,area_harvested,state_x,month,avg_temp,prcp,n_stations,...,prcpZ_m4,prcpZ_m5,prcpZ_m6,prcpZ_m7,prcpZ_m8,prcpZ_m9,prcpZ_m10,prcpZ_m11,prcpZ_m12,log_area
0,AL,1961,WHEAT,26,56000,AL,1,4.365645,80.873333,15,...,-6.919014,-36.864447,65.777775,-17.691224,16.767785,-25.111338,-49.06810,24.515399,171.686542,10.933107
1,AL,1961,WHEAT,26,56000,AL,2,11.385254,317.086667,15,...,-6.919014,-36.864447,65.777775,-17.691224,16.767785,-25.111338,-49.06810,24.515399,171.686542,10.933107
2,AL,1961,WHEAT,26,56000,AL,3,14.444631,215.773333,15,...,-6.919014,-36.864447,65.777775,-17.691224,16.767785,-25.111338,-49.06810,24.515399,171.686542,10.933107
3,AL,1961,WHEAT,26,56000,AL,4,15.149554,114.786667,15,...,-6.919014,-36.864447,65.777775,-17.691224,16.767785,-25.111338,-49.06810,24.515399,171.686542,10.933107
4,AL,1961,WHEAT,26,56000,AL,5,19.999977,71.180000,15,...,-6.919014,-36.864447,65.777775,-17.691224,16.767785,-25.111338,-49.06810,24.515399,171.686542,10.933107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32121,WY,2025,WHEAT,30,89000,WY,8,19.171876,14.586364,22,...,-2.631979,-13.427629,-16.045155,0.349808,-11.022011,-4.148707,15.19078,-13.704891,-4.671743,11.396392
32122,WY,2025,WHEAT,30,89000,WY,9,15.133831,23.554545,22,...,-2.631979,-13.427629,-16.045155,0.349808,-11.022011,-4.148707,15.19078,-13.704891,-4.671743,11.396392
32123,WY,2025,WHEAT,30,89000,WY,10,7.518163,40.604762,21,...,-2.631979,-13.427629,-16.045155,0.349808,-11.022011,-4.148707,15.19078,-13.704891,-4.671743,11.396392
32124,WY,2025,WHEAT,30,89000,WY,11,2.501059,4.363158,19,...,-2.631979,-13.427629,-16.045155,0.349808,-11.022011,-4.148707,15.19078,-13.704891,-4.671743,11.396392
